First, let's install the FastChat (Vicuna) repository.

In [ ]:
!git clone https://github.com/thisserand/FastChat.git
%cd FastChat
!pip install -e .

Cloning into 'FastChat'...
remote: Enumerating objects: 816, done.
remote: Counting objects: 100% (816/816), done.
remote: Compressing objects: 100% (292/292), done.
remote: Total 816 (delta 494), reused 776 (delta 491), pack-reused 0
Receiving objects: 100% (816/816), 27.89 MiB | 26.27 MiB/s, done.
Resolving deltas: 100% (494/494), done.
/content/FastChat/FastChat/FastChat
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/FastChat/FastChat/FastChat
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached transformers-4.28.0.dev0-py3-none-any.whl
  Building editable for fschat (pyproject.toml) ... done
  Created wheel for fschat: filename=fschat-0.1.3-0.editable-py3-none-any.whl size=10647 sha256=5f1be0cf50

Second, we will install the GPTQ-for-LLaMA repository which allows us to use GPTQ quantised models.

In [ ]:
!mkdir repositories
%cd repositories
!git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda
%cd GPTQ-for-LLaMa
!python setup_cuda.py install
%cd ../..

/content/FastChat/FastChat/FastChat/repositories
Cloning into 'GPTQ-for-LLaMa'...
remote: Enumerating objects: 814, done.
remote: Total 814 (delta 0), reused 0 (delta 0), pack-reused 814
Receiving objects: 100% (814/814), 473.43 KiB | 3.88 MiB/s, done.
Resolving deltas: 100% (490/490), done.
/content/FastChat/FastChat/FastChat/repositories/GPTQ-for-LLaMa
running install
/usr/local/lib/python3.9/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
creating quant_cuda.egg-info
writing quant_cuda.egg-info/PKG-INFO
writing dependency_links to quant_cuda.egg-info/dependency_links.txt
writing top-level names to quant_cuda.eg

Okay, now we will download the quantised model weights.

In [ ]:
!python download-model.py anon8231489123/vicuna-13b-GPTQ-4bit-128g

Traceback (most recent call last):
  File "/content/FastChat/FastChat/FastChat/download-model.py", line 196, in <module>
    links, sha256, is_lora = get_download_links_from_huggingface(model, branch)
  File "/content/FastChat/FastChat/FastChat/download-model.py", line 130, in get_download_links_from_huggingface
    fname = dict[i]['path']
KeyError: 0


To be able to run the web UI inside Colab we need virtual terminals. For this, we will use the screen library.

In [ ]:
!sudo apt-get install screen

Reading package lists... Done
Building dependency tree       
Reading state information... Done
screen is already the newest version (4.8.0-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


Launch the controler service in the background.

In [ ]:
!screen -S controller -dm python -m fastchat.serve.controller --host "127.0.0.1"

Launch the model worker in the background.

In [ ]:
!screen -S model_worker -dm python -m fastchat.serve.model_worker --model-path anon8231489123/vicuna-13b-GPTQ-4bit-128g --model-name vicuna-gptq --wbits 4 --groupsize 128 --host "127.0.0.1" --worker-address "http://127.0.0.1:21002" --controller-address "http://127.0.0.1:21001"

Wait a little bit (~30 seconds) before you run this cell to make sure that the model worker is up and running. Using the public URL you can interact with the Vicuna model.

In [ ]:
!python -m fastchat.serve.gradio_web_server --controller-url "http://127.0.0.1:21001" --share